<a href="https://colab.research.google.com/github/arghac14/Customer-Churn-Analysis/blob/master/Clustering(Kmodes_%26_Kprototypes)%5BCustomer_Churn_Analysis%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
url = 'https://raw.githubusercontent.com/SohelRaja/ML_repo_college/master/Decision%20Tree%20with%20Python/WA_Fn-UseC_-Telco-Customer-Churn.csv'
telco = pd.read_csv(url)
telco=telco.drop(['customerID','TotalCharges'] , axis=1)
telco.head()

gender  SeniorCitizen  ... MonthlyCharges Churn
0  Female              0  ...          29.85    No
1    Male              0  ...          56.95    No
2    Male              0  ...          53.85   Yes
3    Male              0  ...          42.30    No
4  Female              0  ...          70.70   Yes

[5 rows x 19 columns]

### Data cleaning and data handling

In [80]:
telco['Partner'].replace(to_replace=[['No'],['Yes']],value=['PartnerNo','PartnerYes'],inplace=True)
telco['Dependents'].replace(to_replace=[['No'],['Yes']],value=['Independent','Dependent'],inplace=True)
telco['PhoneService'].replace(to_replace=[['No'],['Yes']],value=['PSNo','PSYes'],inplace=True)
telco['PaperlessBilling'].replace(to_replace=[['No'],['Yes']],value=['PLBNo','PLBYes'],inplace=True)
telco['MultipleLines'].replace(to_replace=[['No phone service'],['No'],['Yes']],value=['MLNo','MLNo','MLYes'],inplace=True)
telco['OnlineSecurity'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['OSNo','OSNo','OSYes'],inplace=True)
telco['OnlineBackup'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['OBNo','OBNo','OBYes'],inplace=True)
telco['DeviceProtection'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['DPNo','DPNo','DPYes'],inplace=True)
telco['TechSupport'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['TSNo','TSNo','TSYes'],inplace=True)
telco['StreamingTV'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['STVNo','STVNo','STVYes'],inplace=True)
telco['StreamingMovies'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['SMNo','SMNo','SMYes'],inplace=True)
telco.head()

gender  SeniorCitizen  ... MonthlyCharges Churn
0  Female              0  ...          29.85    No
1    Male              0  ...          56.95    No
2    Male              0  ...          53.85   Yes
3    Male              0  ...          42.30    No
4  Female              0  ...          70.70   Yes

[5 rows x 19 columns]

In [81]:
telco['gender'].replace(to_replace=[['Female'],['Male']],value=[0,1],inplace=True)
telco['Partner'].replace(to_replace=[['PartnerNo'],['PartnerYes']],value=[0,1],inplace=True)
telco['Dependents'].replace(to_replace=[['Independent'],['Dependent']],value=[0,1],inplace=True)
telco['PhoneService'].replace(to_replace=[['PSNo'],['PSYes']],value=[0,1],inplace=True)
telco['PaperlessBilling'].replace(to_replace=[['PLBNo'],['PLBYes']],value=[0,1],inplace=True)
telco['MultipleLines'].replace(to_replace=[['MLNo'],['MLYes']],value=[0,1],inplace=True)
telco['OnlineSecurity'].replace(to_replace=[['OSNo'],['OSYes']],value=[0,1],inplace=True)
telco['OnlineBackup'].replace(to_replace=[['OBNo'],['OBYes']],value=[0,1],inplace=True)
telco['DeviceProtection'].replace(to_replace=[['DPNo'],['DPYes']],value=[0,1],inplace=True)
telco['TechSupport'].replace(to_replace=[['TSNo'],['TSYes']],value=[0,1],inplace=True)
telco['StreamingTV'].replace(to_replace=[['STVNo'],['STVYes']],value=[0,1],inplace=True)
telco['StreamingMovies'].replace(to_replace=[['SMNo'],['SMYes']],value=[0,1],inplace=True)
telco['InternetService'].replace(to_replace=[['DSL'],['Fiber optic'],['No']],value=[0,1,2],inplace=True)
telco['PaymentMethod'].replace(to_replace=[['Electronic check'],['Mailed check'],['Bank transfer (automatic)'],['Credit card (automatic)']],value=[0,1,2,3],inplace=True)
telco['Contract'].replace(to_replace=[['Month-to-month'], ['One year'], ['Two year']],value=[0,1,2],inplace=True)
telco['Churn'].replace(to_replace=[['No'], ['Yes']],value=[0,1],inplace=True)
telco.head()

gender  SeniorCitizen  Partner  ...  PaymentMethod  MonthlyCharges  Churn
0       0              0        1  ...              0           29.85      0
1       1              0        0  ...              1           56.95      0
2       1              0        0  ...              1           53.85      1
3       1              0        0  ...              2           42.30      0
4       0              0        0  ...              0           70.70      1

[5 rows x 19 columns]

In [76]:
# #telco['TotalCharges'].apply(pd.to_numeric)
# telco['TotalCharges']=telco['TotalCharges'].astype(str)
# list(telco.TotalCharges)
# telco.TotalCharges.dtype

dtype('O')

In [93]:
df=telco
df_1=df[df['Churn']==1]
df_0=df[df['Churn']==0]
df.describe()

gender  SeniorCitizen  ...  MonthlyCharges        Churn
count  7043.000000    7043.000000  ...     7043.000000  7043.000000
mean      0.504756       0.162147  ...       64.761692     0.265370
std       0.500013       0.368612  ...       30.090047     0.441561
min       0.000000       0.000000  ...       18.250000     0.000000
25%       0.000000       0.000000  ...       35.500000     0.000000
50%       1.000000       0.000000  ...       70.350000     0.000000
75%       1.000000       0.000000  ...       89.850000     1.000000
max       1.000000       1.000000  ...      118.750000     1.000000

[8 rows x 19 columns]

# Clustering

In [23]:
!pip install kmodes

# Kmodes clustering

## Kmode clustering for the dataset where Y=0

In [0]:
num_of_clusters=10

In [83]:
from kmodes.kmodes import KModes


km = KModes(n_clusters=num_of_clusters, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(df_0)

df_0_centroids=km.cluster_centroids_

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1976, cost: 27766.0
Run 1, iteration: 2/100, moves: 415, cost: 27766.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1573, cost: 28532.0
Run 2, iteration: 2/100, moves: 1114, cost: 28430.0
Run 2, iteration: 3/100, moves: 219, cost: 28430.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1669, cost: 28402.0
Run 3, iteration: 2/100, moves: 432, cost: 28402.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1612, cost: 29326.0
Run 4, iteration: 2/100, moves: 388, cost: 29194.0
Run 4, iteration: 3/100, moves: 29, cost: 29194.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1663, cost: 29287.0
Run 5, iteration: 2/100, moves: 46

In [84]:
print("Centroids:\n")
print(df_0_centroids)

Centroids:

[[  0.     0.     0.     0.    72.     1.     0.     1.     0.     1.
    1.     1.     1.     1.     1.     1.     0.   100.3    0.  ]
 [  1.     0.     1.     0.     1.     1.     0.     2.     0.     0.
    0.     0.     0.     0.     1.     1.     1.    19.95   0.  ]
 [  0.     0.     1.     1.    72.     1.     0.     0.     1.     1.
    0.     0.     0.     0.     0.     0.     2.    55.3    0.  ]
 [  0.     0.     0.     0.     3.     1.     1.     1.     0.     0.
    0.     0.     0.     0.     0.     1.     0.    80.55   0.  ]
 [  1.     0.     1.     1.     1.     1.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.    20.     0.  ]
 [  0.     0.     0.     0.     1.     1.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     1.    19.9    0.  ]
 [  1.     0.     0.     0.     1.     1.     0.     2.     0.     0.
    0.     0.     0.     0.     0.     0.     3.    20.05   0.  ]
 [  0.     0.     1.     1.    72.  

## Kmode clustering for the dataset where Y=1

In [85]:
from kmodes.kmodes import KModes


km = KModes(n_clusters=num_of_clusters, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(df_1)

df_1_centroids=km.cluster_centroids_

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 373, cost: 8791.0
Run 1, iteration: 2/100, moves: 173, cost: 8723.0
Run 1, iteration: 3/100, moves: 72, cost: 8716.0
Run 1, iteration: 4/100, moves: 6, cost: 8716.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 690, cost: 8770.0
Run 2, iteration: 2/100, moves: 231, cost: 8705.0
Run 2, iteration: 3/100, moves: 8, cost: 8705.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 608, cost: 8888.0
Run 3, iteration: 2/100, moves: 29, cost: 8888.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 665, cost: 8703.0
Run 4, iteration: 2/100, moves: 236, cost: 8657.0
Run 4, iteration: 3/100, moves: 58, cost: 8657.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, 

In [86]:
print("Centroids:\n")
print(df_1_centroids)

Centroids:

[[  0.     0.     0.     0.     1.     1.     0.     1.     0.     0.
    0.     0.     0.     0.     0.     1.     0.    70.3    1.  ]
 [  0.     1.     0.     0.     4.     1.     1.     1.     0.     1.
    0.     0.     1.     1.     0.     1.     0.   100.2    1.  ]
 [  0.     1.     0.     0.     2.     1.     1.     1.     0.     0.
    1.     0.     1.     1.     0.     1.     0.    99.5    1.  ]
 [  1.     0.     1.     0.     1.     1.     0.     1.     0.     0.
    1.     0.     0.     0.     0.     0.     0.    19.95   1.  ]
 [  1.     0.     1.     0.    66.     1.     1.     1.     0.     1.
    1.     0.     1.     1.     0.     1.     2.    94.85   1.  ]
 [  0.     0.     1.     1.    17.     1.     1.     1.     0.     1.
    1.     0.     1.     1.     0.     1.     0.    94.4    1.  ]
 [  1.     0.     0.     0.     1.     1.     1.     1.     0.     0.
    0.     0.     0.     0.     0.     1.     0.    74.4    1.  ]
 [  1.     0.     0.     0.     1.  

# Kprotoypes clustering

## Kprotoypes clustering for the dataset where Y=0

In [87]:
import numpy as np
from kmodes.kprototypes import KPrototypes


kproto = KPrototypes(n_clusters=num_of_clusters, init='Cao', verbose=2)
clusters = kproto.fit_predict(df_0, categorical=[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,18])


# Print cluster centroids of the trained model.
print(kproto.cluster_centroids_)
# Print training statistics
print(kproto.cost_)
print(kproto.n_iter_)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 1323, ncost: 991010.4445607205
Run: 1, iteration: 2/100, moves: 687, ncost: 926094.0715034514
Run: 1, iteration: 3/100, moves: 446, ncost: 889235.8557968192
Run: 1, iteration: 4/100, moves: 285, ncost: 879032.2037308147
Run: 1, iteration: 5/100, moves: 152, ncost: 875927.0339961856
Run: 1, iteration: 6/100, moves: 80, ncost: 874767.358970841
Run: 1, iteration: 7/100, moves: 14, ncost: 874719.4066283498
Run: 1, iteration: 8/100, moves: 4, ncost: 874711.4193062328
Run: 1, iteration: 9/100, moves: 1, ncost: 874709.9350307517
Run: 1, iteration: 10/100, moves: 0, ncost: 874709.9350307517
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 1161, ncost: 941958.24442736
Run: 2, iteration: 2/100, moves: 446, ncost: 923742.0611903017
Run: 2, iteration: 3/100, moves: 17

In [92]:
for i in range(5):
  print("\nCentroid:",i+1,"\n")
  print("\nCentroid:",i+1,"\n\n(Numeric)\t\t\t\t\t\t(Categorical)\n")
  print(list(np.rint(np.around(kproto.cluster_centroids_[0][i],1))), end=' ')
  print(list(np.rint(np.around(kproto.cluster_centroids_[1][i],1))))
  


Centroid: 1 


Centroid: 1 

(Numeric)						(Categorical)

[40.0, 80.0] [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 2 


Centroid: 2 

(Numeric)						(Categorical)

[5.0, 24.0] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]

Centroid: 3 


Centroid: 3 

(Numeric)						(Categorical)

[47.0, 46.0] [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 4 


Centroid: 4 

(Numeric)						(Categorical)

[17.0, 77.0] [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 5 


Centroid: 5 

(Numeric)						(Categorical)

[22.0, 97.0] [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]


#### Kprototypes clustering for the dataset where Y=1

In [89]:
import numpy as np
from kmodes.kprototypes import KPrototypes


kproto = KPrototypes(n_clusters=num_of_clusters, init='Cao', verbose=2)
clusters = kproto.fit_predict(df_1, categorical=[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,18])


# Print cluster centroids of the trained model.
print(kproto.cluster_centroids_)
# Print training statistics
print(kproto.cost_)
print(kproto.n_iter_)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 399, ncost: 308953.70750731096
Run: 1, iteration: 2/100, moves: 245, ncost: 289349.2946361419
Run: 1, iteration: 3/100, moves: 245, ncost: 260810.1095359313
Run: 1, iteration: 4/100, moves: 164, ncost: 256378.4380572299
Run: 1, iteration: 5/100, moves: 71, ncost: 255400.66361087275
Run: 1, iteration: 6/100, moves: 30, ncost: 255110.17673730792
Run: 1, iteration: 7/100, moves: 18, ncost: 254678.83670537392
Run: 1, iteration: 8/100, moves: 16, ncost: 254350.04038856365
Run: 1, iteration: 9/100, moves: 17, ncost: 254193.3935656778
Run: 1, iteration: 10/100, moves: 9, ncost: 254154.26022982775
Run: 1, iteration: 11/100, moves: 6, ncost: 254137.40008570836
Run: 1, iteration: 12/100, moves: 6, ncost: 254117.71061906402
Run: 1, iteration: 13/100, moves: 5, ncost: 254100.56732669595
Run: 1, iteration: 14/100, moves: 4, ncost: 254095.19253548057
Run: 1, iteration: 15/100, moves: 1, n

In [91]:
for i in range(num_of_clusters):
  print("\nCentroid:",i+1,"\n\n(Numeric)\t\t\t\t\t\t(Categorical)\n")
  print(list(np.rint(np.around(kproto.cluster_centroids_[0][i],1))), end=' ')
  print(list(np.rint(np.around(kproto.cluster_centroids_[1][i],1))))
  


Centroid: 1 

(Numeric)						(Categorical)

[40.0, 80.0] [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 2 

(Numeric)						(Categorical)

[5.0, 24.0] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]

Centroid: 3 

(Numeric)						(Categorical)

[47.0, 46.0] [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 4 

(Numeric)						(Categorical)

[17.0, 77.0] [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 5 

(Numeric)						(Categorical)

[22.0, 97.0] [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 6 

(Numeric)						(Categorical)

[6.0, 49.0] [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

Centroid: 7 

(Numeric)						(Categorical)

[42.0, 102.0] [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0